In [ ]:
import matplotlib.pyplot as plt
import csv
import numpy as np
import matplotlib.patches as patches
import matplotlib
import tifffile
import os
import numpy as np
import sys
import dxchange
import matplotlib
import matplotlib.pyplot as plt
import scipy as sp
import scipy.ndimage as ndimage
%matplotlib inline

In [ ]:
import numpy as np
import tifffile
from holotomocupy.utils import *

iter = 64
bin = 0
ss = 1024//2**bin
nx, ny, nz = 3216//2**bin, 3216//2**bin, 2048//2**bin


In [ ]:
def halfbit(data, center):
    z, y, x = np.indices((data.shape))
    r = np.sqrt((x - center[2])**2 + (y - center[1])**2 + (z - center[0])**2)
    r=r.astype(np.int32)
    nr = np.bincount(r.ravel())
    return (0.2071+1.9102/np.sqrt(nr))/(1.2071+0.9102/np.sqrt(nr)) 

def radial_profile(data, center):
    z, y, x = np.indices((data.shape))
    r = np.sqrt((x - center[2])**2 + (y - center[1])**2 + (z - center[0])**2)
    r=r.astype(np.int32)
    

    tbinre = np.bincount(r.ravel(), data.real.ravel())
    tbinim = np.bincount(r.ravel(), data.imag.ravel())
    
    nr = np.bincount(r.ravel())
    radialprofile = (tbinre+1j*tbinim) / np.sqrt(nr)
    
    return radialprofile
from scipy.signal import savgol_filter

def smooth_savgol(data, window_size, poly_order):
    if window_size % 2 == 0 or window_size < 1:
      raise ValueError("Window size must be an odd positive integer")
    if poly_order >= window_size:
       raise ValueError("poly_order must be less than window_size")
    data = np.pad(data,(32,32),'edge')
    data = savgol_filter(data, window_size, poly_order)
    data=data[32:-32]
    return data

In [ ]:

wsize = ss

# ff1 = np.zeros([2*wsize,2*wsize,2*wsize],dtype='complex64')
# hbit = halfbit(ff1,np.array([2*wsize,2*wsize,2*wsize])//2)
hbit = np.load('hbit.npy')
len(hbit)


In [ ]:
import matplotlib as mpl

mpl.rcParams.update({
    "font.family": "serif",
    "font.serif": ["Times New Roman", "Times", "DejaVu Serif"],  # fallbacks
})

mpl.rcParams.update({
    "font.family": "serif",
    "font.serif": ["Times New Roman", "Times", "DejaVu Serif"],
    "mathtext.fontset": "stix",   # Times-like math
    # optional:
    # "mathtext.default": "regular",
})
mpl.rcParams['font.size'] = 20  # optional
mpl.rcParams["xtick.labelsize"] = 20
mpl.rcParams["ytick.labelsize"] = 20

In [ ]:

fig, ax = plt.subplots(1,1,figsize=(10,5))


frc2=np.load(f'frc2{iter}.npy')
pfrc2=np.load('pfrc2.npy')
frc2c=smooth_savgol(frc2, 31, 5)
pfrc2c=smooth_savgol(pfrc2, 31, 5)
hbit=np.load('hbit.npy')


a = np.hamming(2*len(pfrc2c))
a = a[len(a)//2:]
# plt.plot(a)
pfrc2c*=a
pfrc2c=np.abs(pfrc2c)[:wsize]
a = np.hamming(2*len(pfrc2c))
a = a[len(a)//2:]
# plt.plot(a)
pfrc2c*=a
pfrc2c=np.abs(pfrc2c)[:wsize]

# a = np.hamming(2*len(pfrc2c))
# a = a[len(a)//2:]
# frc2c*=a
frc2c=np.abs(frc2c)[:wsize]

ax.plot(frc2c[:].real,linewidth=1.5, label=f'proposed')
ax.plot(pfrc2c[:].real,linewidth=1.5, label=f'conventional')

ax.plot(hbit[:wsize],linewidth=1.5,label='1/2-bit')

ax.grid()
ax.set_ylim([0,1])
lg = plt.legend(loc="upper right")
lg.get_title().set_fontsize(16)
plt.ylabel('FSC',rotation=90)
ax2 = ax.secondary_xaxis('top')
ax.set_xlabel(f'Spatial frequency (1/$\mu$m)')
ax2.set_xlabel('Spatial resolution (nm)')
ax.set_xticks(np.linspace(0,wsize,11),np.int32(np.linspace(0,1/(2*20e-9),11)/1e6))
ax2.set_xticks(np.linspace(0,wsize,11),(np.int32(2000*2**bin/np.linspace(1,10,11))/10))
# # plt.show()
plt.savefig(f'figs/{iter}_{wsize}.png',dpi=300, bbox_inches="tight", pad_inches=0.02)

In [ ]:
128e-6/40e-9